# SEN163A - Fundamentals of Data Analytics
# Assignment 2 - Large-scale Internet Data Analysis
### Ir. Jacopo De Stefani - [J.deStefani@tudelft.nl](mailto:J.deStefani@tudelft.nl)
### Joao Pizani Flor, M.Sc. - [J.p.pizaniflor@tudelft.nl](mailto:J.p.pizaniflor@tudelft.nl)

### 05-03-2022
## Group 2
- Emmanuel M Boateng - '5617642'
- Joost Oortwijn - '4593472'
- Philip Busscher - ''4611993''
- Floris Kool - ''4975243''


# Introduction
This notebook presents the results of our analysis of potential ASN's which can be used for hosting the mobile banking IT infrastructure for GNI Bank. Based on three datasets (described in chapter 1), we have idenified 4 locations which are, based on the available data, the best hosting options for GNI Bank. The indepth analysis consists of 4 parts and is shown in chapter 2. The final conclusion of our analysis is presented in chapter 3. In addition, we have identified different limitations to the available data which could decrease the usability of our recommendations in chapter 3, therefore these should be taken into account. These limitations are described in chapter 1.2.

# 1. Dataset description

Short description of the 4 datasets used

## 1.1 Opening the data

In [12]:
import pickle
import time
import bz2
import os
import sys
import json
import pandas as pd
import ipaddress
import io

In [5]:
#AS Dataset

AS_Filename = 'data/AS_dataset.pkl'

with open(AS_Filename, 'rb') as file:
    
    AS_df = pickle.load(file)

In [6]:
#Probe dataset

Probe_Filename = 'data/probe_dataset.pkl'

with open(Probe_Filename, 'rb') as file:
    
    P_df = pickle.load(file)

In [16]:
#Ripe dataset (Single file)

#Option 1 decompressed file
decomFilename = 'data/ping-2022-03-01T2300_decom'
#decomFile     = open(decomFilename, 'rt')

#Option 2 BZ2 file
bz2Filename = 'D:/FoDa Data/ping-2022-03-01T0000.bz2'
bz2File     = bz2.open(bz2Filename, 'rt')


# List of tuples
# https://stackoverflow.com/questions/28056171/how-to-build-and-fill-pandas-dataframe-from-for-loop
tuple_list = []

start  = time.time()
i = 0

#for line in bz2File:
for line in bz2File:
    
    decoded_line = json.loads(line)
    if "af" in decoded_line and "dst_addr" in decoded_line and "prb_id" in decoded_line and "avg" in decoded_line: 
        if decoded_line["af"] == 4:
            tuple_list.append((decoded_line["dst_addr"],decoded_line["prb_id"],decoded_line["avg"]))
            
    i = i + 1
    if i > 1000000:
        break

            
dur         = round(time.time() - start,2)
print("Loading took: "  + str(dur) + " seconds")
print("Lines added to tuple: " + str(len(tuple_list)))

#finally close bz2File
bz2File.close()

Loading took: 20.75 seconds
Lines added to tuple: 601435


In [17]:
#Load tuples data into dataframe
start  = time.time()

RIPE_df = pd.DataFrame(tuple_list)

dur         = round(time.time() - start,2)
print("Loading took: "  + str(dur) + " seconds")

Loading took: 0.22 seconds


In [18]:
#IP 2 Location dataset

IP_Filename = "data/IP2LOCATION-LITE-DB1.CSV"

ipv4_df = pd.read_csv(IP_Filename)

ipv4_df.rename(columns = {'0':'ip_from', '16777215':'ip_to',
                              '-':'country_code','-.1':'country_name'}, inplace = True)

More detailed description of data if needed (Can also be after opening each dataset)

## 1.2 Limitations in data (Question A)

Evaluate if there are limitations in the provided datasets (AS and probe data set). If you find limitations, describe these and conjecture possible reasons, supported with data.

### 1.2.1 Limitations in the AS and Probe dataset
A limitation in the AS and probe dataset is the fact that according to the AS dataset there are 534 AS's that can be used for hosting in the EU. But when the AS data set is merged with the probe dataset the number of potential AS's has decreased to 339. An explanation for this is the fact that the probe dataset consists of significantly less AS's then the AS dataset (3652 compared to 60122). This can be seen as limitation as potential AS's are left out of the analysis eventhough these could be a interesting options for hosting the EU. 

In [19]:
len(AS_df.loc[(AS_df['type'] == 'hosting') & (AS_df['Country'].isin(EU_list))])

534

In [20]:
as_probe_joined_df = pd.merge(P_df,AS_df, on='ASN')
len(as_probe_joined_df.loc[(as_probe_joined_df['type'] == 'hosting') & (as_probe_joined_df['Country'].isin(EU_list))])

339

In [21]:
AS_df['ASN'].value_counts()

AS55330     1
AS31573     1
AS200609    1
AS200605    1
AS51879     1
           ..
AS48290     1
AS48295     1
AS48303     1
AS48360     1
AS37485     1
Name: ASN, Length: 60122, dtype: int64

In [22]:
P_df['ASN'].value_counts()

AS3320      351
AS7922      334
AS6830      333
AS3215      243
AS12322     210
           ... 
AS8282        1
AS12775       1
AS199484      1
AS199853      1
AS49432       1
Name: ASN, Length: 3652, dtype: int64

### 1.2.2 Limitation in the IP location dataset

### 1.2.3 Limitations in the RIPE dataset

When looking at the RIPE dataset, it was found that for some lines the IP destination addresses were missing. This was not the case for the Probe ID's and average round trip times as these were always included within the lines of the RIPE dataset. Below the number of missing values in the first 5 million lines are shown. When looking at the lines where the destination address is missing, it can be seen that these adresses are missing because there is no node or server name provided or known. Obviously this results in the lack of a destination address of that specific ping measurement. Due to the size of the available dataset this is not assumed as a limitation and the lines with missing destination address are simply skipped when opening the RIPE dataset. 

In [ ]:
bz2Filename = 'data/ping-2022-03-01T0000.bz2'
bz2File_limitation = bz2.open(bz2Filename, 'rt') 
missing_adres = 0
missing_probeID = 0
missing_avg = 0
line_number = 0

for line in bz2File_limitation:
    decoded_line = json.loads(line)
    line_number += 1
    if "dst_addr" not in decoded_line: 
        missing_adres += 1
      
    if "prb_id" not in decoded_line: 
        missing_probeID += 1
        
    if "avg" not in decoded_line: 
        missing_avg += 1
           
    if line_number > 5000000:
        print('There are', missing_adres, 'missing IP destination addresses in the first 5m lines of the RIPE dataset (for one hour)')
        print('There are', missing_probeID, 'missing probe ID\'s in the first 5m lines of the RIPE dataset (for one hour)')
        print('There are', missing_avg, 'missing average round-trip time values in the first 5m lines of the RIPE dataset (for one hour)')
        break
        
bz2File_limitation.close()

In [ ]:
bz2Filename = 'data/ping-2022-03-01T0000.bz2'
bz2File_limitation = bz2.open(bz2Filename, 'rt') 
line_number = 0

for line in bz2File_limitation:
    decoded_line = json.loads(line)
    line_number += 1
    if "dst_addr" not in decoded_line: 
        print("Destination adress is missing in at line:", line_number,':')
        print (line)
    
    if line_number > 10000:
        break

bz2File_limitation.close()

# 2 Analysis

Short description of what is going to be analyzed

## 2.1 AS (Question B)

With the AS and probe data set, find the number m of AS’s that can be used for hosting in the EU
and have probes in the RIPE data set. Sort the ASN’s in ascending order and include the first and last
three in your report (number, name and country).


In [23]:
#Merge the AS and Probe datasets
as_probe_joined_df = pd.merge(P_df,AS_df, on='ASN')

In [24]:
# EU country codes retrieved from: https://ec.europa.eu/eurostat/statistics-explained/index.php?title=Glossary:Country_codes
EU_list = ['BE','BG','CZ','DK','DE','EE','IE','EL','ES','FR','HR','IT','CY','LV','LT','LU','HU','MT','NL','AT','PL','PT','RO','SI','SK','FI','SE']

# Filter data set for AS's that can be used for hosting in the EU
as_probe_joined_df = as_probe_joined_df.loc[(as_probe_joined_df['type'] == 'hosting') & (as_probe_joined_df['Country'].isin(EU_list))]

In [25]:
#Get the unique number of probe IDs that are in the RIPE Data
unique_prbID = RIPE_df[1].unique()

print("Unique probe IDs: " + str(len(unique_prbID)))

Unique probe IDs: 11543


In [26]:
#Filter the data set by only selecting the ASN's that have probes in the Ripe dataset
AS_Probe_RIPE_df = as_probe_joined_df.loc[as_probe_joined_df['prb_id'].isin(unique_prbID)]

#Sort by ASN
AS_Probe_RIPE_df.sort_values(by=['ASN']).sort_values(by=['ASN'])

print("Number of probes connected to AS that can be used for hosting in the EU and are in the RIPE dataset: " + str(len(AS_Probe_RIPE_df["ASN"])))


Number of probes connected to AS that can be used for hosting in the EU and are in the RIPE dataset: 234


In [27]:
#Remove duplicate ASNs (Probes connected to same AS)
display_df = AS_Probe_RIPE_df.drop_duplicates(subset=['ASN'])

#Remove unused columns
display_df = display_df.drop(columns=['prb_id', 'NumIPs', 'type'])

#Print anwser to question B
print("Number of AS that can be used for hosting in the EU and are in the RIPE dataset: " + str(len(display_df["ASN"])))


Number of AS that can be used for hosting in the EU and are in the RIPE dataset: 113


In [74]:
#First 3 probes
display_df.head(3)

,ASN,Country,Name
2451,AS12859,NL,BIT BV
2523,AS25596,NL,Cambrium IT Services B.V.
2844,AS15598,DE,QSC AG


In [75]:
#Last 3 probes
display_df.tail(3)

,ASN,Country,Name
10498,AS47692,AT,Nessus GmbH
10589,AS30823,DE,combahton GmbH
10664,AS51167,DE,Contabo GmbH


Description of results

## 2.2 Hosting location (Question C)
For a single hour in the RIPE data set: find all valid entries where the probe has hosting type AS and
the target IPv4 is from an EU country. Implement this in an efficient way.

In [28]:
#Selects all entries in RIPE data with probe connected to EU as of type hosting
RIPE_HostAS_df = RIPE_df.loc[RIPE_df[1].isin(AS_Probe_RIPE_df['prb_id'])]

print("Entries with probe connected to an EU as with type hosting: " + str(len(RIPE_HostAS_df[1])))

Entries with probe connected to an EU as with type hosting: 24896


In [29]:
#Convert IP strings to IP integers
for i in RIPE_HostAS_df.index:
    
    IP_Splitstring = RIPE_HostAS_df[0][i].split(".") 
    RIPE_HostAS_df.at[i, 0] = int(IP_Splitstring[0]) * 16581375 + int(IP_Splitstring[1]) * 65025 + int(IP_Splitstring[2]) * 255 + int(IP_Splitstring[3])



#Add Integer values of IP to dataframe
#RIPE_HostAS_df["IP_Integer"] = IPs_Integer

156.154.79.254


In [64]:
#Sorting the IP lists so we can check from low to high IPs
RIPE_HostAS_df = RIPE_HostAS_df.sort_values(by=[0])
ipv4_df = ipv4_df.sort_values(by=["ip_from"])

Dest_Addr_Countries = []
ripeindex = 0
ipindex = 0

#Check if there are IP addresses lower than included in the IP2Location dataset
while RIPE_HostAS_df.iat[ripeindex, 0] < ipv4_df.at[ipindex, "ip_from"]:
    ripeindex = ripeindex + 1
    Dest_Addr_Countries.append("-")

print("IP addresses not included in IP2location dataset: " + str(ripeindex))

#Check for each range of IP addresses in the IP2Location dataset which dst_addr IPs are present
#Break loop early if the length of the RIPE dataset is reached
for ipindex in ipv4_df.index:
    while RIPE_HostAS_df.iat[ripeindex, 0] >= ipv4_df.at[ipindex, "ip_from"] and RIPE_HostAS_df.iat[ripeindex, 0] <= ipv4_df.at[ipindex, "ip_to"]:
        Dest_Addr_Countries.append(ipv4_df.at[ipindex, "country_code"])
        ripeindex = ripeindex + 1
        if ripeindex >= len(RIPE_HostAS_df[0]):
            break
    
    if ripeindex >= len(RIPE_HostAS_df[0]):
        break

print("IP addresses linked to country: " + str(len(Dest_Addr_Countries)))
#Add list for destination address location to dataframe
RIPE_HostAS_df["Country"] = Dest_Addr_Countries



IP addresses not included in IP2location dataset: 4
IP addresses linked to country: 24896


In [128]:
#Remove entries not in EU
RIPE_HostAS_df = RIPE_HostAS_df.loc[RIPE_HostAS_df['Country'].isin(EU_list)]

print("Entries with probe connected to an EU AS with type hosting and destination address within EU: " + str(len(RIPE_HostAS_df[1])))

Entries with probe connected to an EU AS with type hosting and destination address within EU: 133653


In [65]:
RIPE_HostAS_df.head(10)

,0,1,2,Country
1,123,26262,3.964487,-
24655,16581376,21831,4.715625,-
182742,16646656,1000013,12.297646,-
319075,16646656,1000013,10.942383,-
392390,36807144,6726,-1.000000,AE
192059,36807144,6667,-1.000000,AE
439497,36807144,6734,-1.000000,AE
55541,36807144,6509,-1.000000,AE
55540,36807144,6509,-1.000000,AE
567533,36807144,6726,-1.000000,AE


## Part C Alternative Approach reading all the dataset

In [ ]:
import pickle
import time
import bz2
import os
import sys
import json
import pandas
import io
import datetime
import socket
import struct

def ip2int(addr):
    return struct.unpack("!I", socket.inet_aton(addr))[0]

with open('data/AS_dataset.pkl', 'rb') as file:
    AS_df = pickle.load(file)
    
with open('data/probe_dataset.pkl', 'rb') as file:    
    P_df = pickle.load(file)
    
decomFilename = 'data/ping-2022-03-01T2300.bz2'
#decomFile     = bz2.open(decomFilename, 'rt')   
merged_df = P_df.merge(AS_df)

ipv4_df = pandas.read_csv("data/IP2LOCATION-LITE-DB1.CSV")
ipv4_df.rename(columns = {'0':'ip_from', '16777215':'ip_to',
                              '-':'country_code','-.1':'country_name'}, inplace = True)


EU_Countries = ["AT","BE","HR","CY","CZ","DK","EE","FI","FR","GR","DE","HU",
                "IE","IT","LV","LT","LU","MT","NL","PL","PT","RO","SK","SI",
                "ES","SE"]

EU_data = merged_df[merged_df['Country'].isin(EU_Countries)]
EU_Hosting = EU_data[EU_data['type'] == 'hosting']



merged_df.insert(2, 'AS', merged_df['ASN'].str.replace('AS',''))
merged_df['AS'] = pandas.to_numeric(merged_df['AS'])
merged_df['prb_id'] = pandas.to_numeric(merged_df['prb_id'])


merged_df_sorted = merged_df.sort_values('AS')
df_HostingAS = merged_df[merged_df['type'] == 'hosting']

ipv4_df.head()
tpl = ipv4_df.loc[:, 'ip_from':'ip_to'].apply(tuple, 1).tolist()
idx = pandas.IntervalIndex.from_tuples(tpl, 'both')

t0 = time.time()
time.sleep(0.000001)
with open(decomFilename, 'rb') as file:
    decomp = bz2.BZ2Decompressor()
    residue = b''
    total_lines = 0
    m = 0
    checked = []
    #102400 Bytes = 102.4 KB (in decimal)
    #102400 Bytes = 100 KB (in binary)
    #Iterate over RIPE data in  100 KB chunks 
    for data in iter(lambda: file.read(100 * 1024), b''):
        # process the raw data and  concatenate residual of the previous block 
        #to the beginning of the current raw data block
        raw = residue + decomp.decompress(data) 
        residue = b''
        ## process_data(current_block) => do the processing of the 
        ##current data block
        current_block = raw.split(b'\n')
        if raw[-1] != b'\n':
            residue = current_block.pop() # last line could be incomplete
        ##Process all data in the current block to check    
        for items in current_block:
            df_dict = json.loads(items.decode('utf-8'))
            if ('dst_addr' in df_dict) and (df_dict['af'] == 4):# and (ip2int(df_dict['dst_addr'])>0:
                ##convert to interger
                df_ip = ip2int(df_dict['dst_addr'])
                
                if df_ip > 0: #and (df_dict['prb_id'] not in checked)): # certain lines have 0.0.0.0 IP
                    loc = idx.get_loc(df_ip)
                    if ((ipv4_df.loc[loc,'country_code'] in EU_Countries) and (df_dict['prb_id'] not in checked)):
                        #if len(EU_Hosting[EU_Hosting['prb_id'] == df_dict['prb_id']])!=0:
                            #print(df_HostingAS[df_HostingAS['prb_id'] == df_dict['prb_id']])
                        m +=1 ## increment count
                       ##create a list of probes that could be used later                     
                        checked.append(df_dict['prb_id']) 
        total_lines += len(current_block)
    total_lines += 1

print("Total processing time: ",(time.time() - t0))
print("Total number of probe entries with hosting type AS and EU target in RIPE is %i" %(m))
fi.close()



Description of results

## 2.3 Latency (Question D)
Move from using only an hour to the full day. It is advisable to store the raw results of each file. Then,
using all processed files, calculate the average latency’s for each country-AS combination and store
the results into one ncountries ×m matrix. If we could place one server in each country, what would the
minimum average latency be for each country? (include in your report)


In [132]:
#We want a matrix of 26 countries * 113 ASNs (For a single file, should be more for 24 files)
import numpy as np
import pickle
import time
import bz2
import os
import sys
import json
import pandas
import io
import datetime
import socket
import struct
from ip2geotools.databases.noncommercial import HostIP #gets the country code from ip

def ip2int(addr):
    return struct.unpack("!I", socket.inet_aton(addr))[0]


with open('data/AS_dataset.pkl', 'rb') as file:
    AS_df = pickle.load(file)
    
with open('data/probe_dataset.pkl', 'rb') as file:    
    P_df = pickle.load(file)
    
decomFilename = 'data/ping-2022-03-01T2300.bz2'
#decomFile     = bz2.open(decomFilename, 'rt')   
merged_df = P_df.merge(AS_df)
probes = merged_df['prb_id'].tolist()
print(type(probes))

ipv4_df = pandas.read_csv("data/IP2LOCATION-LITE-DB1.CSV")
ipv4_df.rename(columns = {'0':'ip_from', '16777215':'ip_to',
                              '-':'country_code','-.1':'country_name'}, inplace = True)


EU_Countries = ["AT","BE","HR","CY","CZ","DK","EE","FI","FR","GR","DE","HU",
                "IE","IT","LV","LT","LU","MT","NL","PL","PT","RO","SK","SI",
                "ES","SE"]

EU_data = merged_df[merged_df['Country'].isin(EU_Countries)]
EU_Hosting = EU_data[EU_data['type'] == 'hosting']

merged_df.insert(2, 'AS', merged_df['ASN'].str.replace('AS',''))
merged_df['AS'] = pandas.to_numeric(merged_df['AS'])
merged_df['prb_id'] = pandas.to_numeric(merged_df['prb_id'])

df_HostingAS = merged_df[merged_df['type'] == 'hosting']

ipv4_df.head()
tpl = ipv4_df.loc[:, 'ip_from':'ip_to'].apply(tuple, 1).tolist()
idx = pandas.IntervalIndex.from_tuples(tpl, 'both')
    
l =pandas.date_range('2022-03-01', periods=24, freq='60min').strftime('data/ping-%Y-%m-%dT%H%M.bz2').tolist()
#print(l)
m = 0
data = []
df = []
for dataset in l:
    decomFilename = dataset
    print(decomFilename)
    tstamp = str(decomFilename.strip('.bz2')[-5:])
    #print(tstamp)

    with open(decomFilename, 'rb') as fi:
        decomp = bz2.BZ2Decompressor()
        residue = b''
        total_lines = 0
        
        for data in iter(lambda: fi.read(100 * 1024), b''):
            raw = residue + decomp.decompress(data) # process the raw data and  concatenate residual of the previous block to the beginning of the current raw data block
            residue = b''
            # process_data(current_block) => do the processing of the current data block
            current_block = raw.split(b'\n')
            if raw[-1] != b'\n':
                residue = current_block.pop() # last line could be incomplete
            for items in current_block:
                df_dict = json.loads(items.decode('utf-8'))
                if ('dst_addr' in df_dict and df_dict['dst_addr']!= '0.0.0.0' and df_dict['af'] == 4 and df_dict['avg']>0):
                    df_ip = ip2int(df_dict['dst_addr'])
                    loc = idx.get_loc(df_ip)
                    if ((ipv4_df.loc[loc,'country_code'] in EU_Countries) and df_dict['prb_id'] in probes):
                        res = {key: df_dict[key] for key in df_dict.keys()
                                   & {'avg','prb_id'}}
                        res['Country'] = ipv4_df.loc[loc,'country_code']
                        res['t'] = tstamp
                        #print(res)
                        df_line = pandas.DataFrame.from_dict(res, orient='index')
                        df.append(df_line)
                
            total_lines += len(current_block)
        total_lines += 1
        if total_lines > 1000:
            print('i should take a break')
            break
            
    df.groupby('Country', as_index=False)['avg'].mean() 
    m+=1
    if m>=1:
        print('Taking a break')
        break
df.head()


,0,1,2
0,BE,AS12859,85.256193
1,BE,AS25596,-1.000000
2,BE,AS15598,NaN
3,BE,AS203953,12.915613
4,BE,AS51815,3.754592


In [ ]:
#tuple list
test1= []

for country in EU_list:
    
    #Filter each country's ping values seperately into a dataframe
    country_df = RIPE_HostAS_df.loc[RIPE_HostAS_df['Country'] == country]
    
    for ASN in display_df["ASN"]:
        
        #Filter probe IDs for each seperate ASN
        #There are more probes than ASs to calculate the average ping more accurately we use all probes
        prb_df = AS_Probe_RIPE_df.loc[AS_Probe_RIPE_df['ASN'] == ASN]                            
        
        #Filter the ping data so it includes all probes from selected ASN and selected country
        temp_df = country_df.loc[country_df[1].isin(prb_df['prb_id'])]
        
        #Create sum of all ASN - Country ping measurements
        sumvalue = 0
        i = 0
        for pingvalue in temp_df[2]:
            sumvalue = sumvalue + pingvalue
            i = i+1
        
        #Check if there are ping measurements between AS - Country
        #Calculate average when needed, enter '-' when no data available
        if not i == 0:
            average = sumvalue/i
            test1.append((country, ASN, average))
        else:
            test1.append((country, ASN, np.nan))

#Load tuple list into dataframe
test1_df = pd.DataFrame(test1)        
test1_df.head(5)

In [150]:
test1_df.columns = ['Country','ASN','Average latency']

In [297]:
test1_df = test1_df.iloc[:, 1:] # asn and latency
df_groupby = test1_df.groupby('ASN')['Average latency'].apply(list)

In [298]:
new_dftesttest = np.zeros((len(df_groupby), len(df_groupby[0])))
for i in range(len(df_groupby)):
    for j in range(len(df_groupby[0])):
        new_dftesttest[i,j] = df_groupby[i][j]

df_groupby.index
final_df = pd.DataFrame(new_dftesttest.transpose())   

In [299]:
column_list = list(df_groupby.index)

In [300]:
final_df.columns=column_list

In [301]:
final_df['index']=EU_list

In [302]:
final_df.set_index('index')

,AS12676,AS12824,AS12859,AS12876,AS12993,AS13287,AS15401,AS15598,AS15685,AS15817,...,AS61211,AS62000,AS62282,AS62416,AS6724,AS8304,AS8315,AS8560,AS8893,AS9211
index,,,,,,,,,,,,,,,,,,,,,
BE,88.093683,17.702773,85.256193,NaN,10.643335,NaN,NaN,NaN,226.285196,60.362766,...,NaN,76.313670,13.343822,92.592569,83.259170,NaN,NaN,80.875999,87.930196,33.442691
BG,-1.000000,NaN,-1.000000,NaN,NaN,NaN,NaN,NaN,NaN,-1.000000,...,NaN,-1.000000,NaN,-1.000000,-1.000000,NaN,NaN,-1.000000,-1.000000,-1.000000
CZ,32.819706,27.023314,26.958519,6.447432,47.118373,44.372507,32.211990,13.965507,7.243503,32.663917,...,15.138536,50.525012,30.112869,60.593795,39.976036,113.530596,21.579390,32.521064,39.861434,11.843888
DK,34.891794,28.365377,38.888919,15.644184,36.195367,37.156879,33.930247,4.563901,12.173853,38.981792,...,25.813521,48.207860,27.269326,60.481178,47.970564,34.615074,12.643969,37.872808,44.412661,25.870933
DE,49.938872,47.639296,48.532796,76.007365,54.801323,16.256942,18.103846,5.549774,9.584655,45.951487,...,22.065525,58.350937,11.998714,72.510263,55.345841,58.742821,21.460551,46.526449,55.097224,30.355043
EE,18.173319,NaN,14.366154,NaN,NaN,NaN,NaN,NaN,NaN,20.789540,...,NaN,23.625998,NaN,54.242222,23.667669,NaN,NaN,22.875320,6.618570,NaN
IE,289.156367,NaN,247.740473,NaN,NaN,231.463974,NaN,NaN,NaN,200.267857,...,160.178071,215.952162,NaN,222.628365,296.038515,NaN,NaN,291.453679,294.540788,NaN
EL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ES,54.128359,NaN,53.365201,-1.000000,72.030021,103.286695,NaN,NaN,NaN,45.791166,...,53.363730,125.304564,NaN,92.223453,62.043651,34.328491,NaN,73.916970,60.400514,-1.000000


## Reading data for D

In [78]:
RIPE_Filenames = pd.date_range('2022-03-01', periods=24, freq='60min').strftime('D:/FoDa Data/ping-%Y-%m-%dT%H%M.bz2').tolist()

Complete_ASN_List = []
Complete_RIPE_Entries_df = pd.DataFrame(
                                        {
                                            0: [],
                                            1: [],
                                            2: [],
                                            'Country': []
                                        })

for filename in RIPE_Filenames:
    #Read RIPE data
    print(filename)
    bz2File     = bz2.open(filename, 'rt')
    
    tuple_list = []
    
    start  = time.time()
    i = 0
    
    for line in bz2File:
        decoded_line = json.loads(line)
        if "af" in decoded_line and "dst_addr" in decoded_line and "prb_id" in decoded_line and "avg" in decoded_line: 
            if decoded_line["af"] == 4:
                tuple_list.append((decoded_line["dst_addr"],decoded_line["prb_id"],decoded_line["avg"]))
        #i = i + 1
        #if i > 1000000:
        #    break;
    
    bz2File.close()
    
    RIPE_df = pd.DataFrame(tuple_list)
    
    dur         = round(time.time() - start,2)
    print("Lines added: " + str(len(tuple_list)) + "Loading took: " + str(dur) + " seconds")
    
    #Get list of ASNs
    unique_prbID = RIPE_df[1].unique()
    
    AS_Probe_RIPE_df = as_probe_joined_df.loc[as_probe_joined_df['prb_id'].isin(unique_prbID)]
    
    unique_ASNs = AS_Probe_RIPE_df['ASN'].unique()
    Complete_ASN_List.extend(unique_ASNs)
    
    
    #Get RIPE entries with dst addr in eu
    RIPE_HostAS_df = RIPE_df.loc[RIPE_df[1].isin(AS_Probe_RIPE_df['prb_id'])]
    
    for i in RIPE_HostAS_df.index:
        RIPE_HostAS_df.at[i, 0] = int(ipaddress.IPv4Address(RIPE_HostAS_df[0][i]))

    RIPE_HostAS_df = RIPE_HostAS_df.sort_values(by=[0])
    ipv4_df = ipv4_df.sort_values(by=["ip_from"])

    Dest_Addr_Countries = []
    ripeindex = 0
    ipindex = 0

    while RIPE_HostAS_df.iat[ripeindex, 0] < ipv4_df.at[ipindex, "ip_from"]:
        ripeindex = ripeindex + 1
        Dest_Addr_Countries.append("-")

    for ipindex in ipv4_df.index:
        while RIPE_HostAS_df.iat[ripeindex, 0] >= ipv4_df.at[ipindex, "ip_from"] and RIPE_HostAS_df.iat[ripeindex, 0] <= ipv4_df.at[ipindex, "ip_to"]:
            Dest_Addr_Countries.append(ipv4_df.at[ipindex, "country_code"])
            ripeindex = ripeindex + 1
            if ripeindex >= len(RIPE_HostAS_df[0]):
                break

        if ripeindex >= len(RIPE_HostAS_df[0]):
            break
            
    RIPE_HostAS_df["Country"] = Dest_Addr_Countries
    RIPE_HostAS_df = RIPE_HostAS_df.loc[RIPE_HostAS_df['Country'].isin(EU_list)]    
    
    
    #Add entries to complete dataframe
    frames = [Complete_RIPE_Entries_df, RIPE_HostAS_df]
    Complete_RIPE_Entries_df = pd.concat(frames)
    
    dur         = round(time.time() - start,2)
    print("Added " + str(len(RIPE_HostAS_df[0])) + " entries and " + str(len(unique_ASNs)) + " ASNs in " + str(dur) + " seconds")
    print()
    
    break

#Remove duplicates
Complete_ASN_Set = set(Complete_ASN_List)


D:/FoDa Data/ping-2022-03-01T0000.bz2
Added 286373 entries and 113 ASNs in 742.29 seconds



## Same code, but reading data raw

In [81]:
#Changed 24 -> 6
RIPE_Filenames = pd.date_range('2022-03-01', periods=24, freq='60min').strftime('D:/FoDa Data/ping-%Y-%m-%dT%H%M.bz2').tolist()

Complete_ASN_List = []
Complete_RIPE_Entries_df = pd.DataFrame({0:[], 1:[], 2:[], 'Country':[]})


for filename in RIPE_Filenames:
    #Read RIPE data
    print(filename)
    
    start  = time.time()
    with open(filename, 'rb') as fi:
        decomp = bz2.BZ2Decompressor()
        residue = b''
        total_lines = 0
        m = 0
        tuple_list = []
    
        for data in iter(lambda: fi.read(100 * 1024), b''):
            raw = residue + decomp.decompress(data) # process the raw data and  concatenate residual of the previous block to the beginning of the current raw data block
            residue = b''
            # process_data(current_block) => do the processing of the current data block
            current_block = raw.split(b'\n')
            if raw[-1] != b'\n':
                residue = current_block.pop() # last line could be incomplete

            for items in current_block:
                df_dict = json.loads(items.decode('utf-8'))
                if ('dst_addr' in df_dict) and (df_dict['af'] == 4):
                    tuple_list.append((df_dict["dst_addr"],df_dict["prb_id"],df_dict["avg"]))
    
    fi.close()
    
    RIPE_df = pd.DataFrame(tuple_list)
    
    #Get list of ASNs
    unique_prbID = RIPE_df[1].unique()
    
    AS_Probe_RIPE_df = as_probe_joined_df.loc[as_probe_joined_df['prb_id'].isin(unique_prbID)]
    
    unique_ASNs = AS_Probe_RIPE_df['ASN'].unique()
    Complete_ASN_List.extend(unique_ASNs)
    
    
    #Get RIPE entries with dst addr in eu
    RIPE_HostAS_df = RIPE_df.loc[RIPE_df[1].isin(AS_Probe_RIPE_df['prb_id'])]
    
    for i in RIPE_HostAS_df.index:
        RIPE_HostAS_df.at[i, 0] = int(ipaddress.IPv4Address(RIPE_HostAS_df[0][i]))

    RIPE_HostAS_df = RIPE_HostAS_df.sort_values(by=[0])
    ipv4_df = ipv4_df.sort_values(by=["ip_from"])

    Dest_Addr_Countries = []
    ripeindex = 0
    ipindex = 0

    while RIPE_HostAS_df.iat[ripeindex, 0] < ipv4_df.at[ipindex, "ip_from"]:
        ripeindex = ripeindex + 1
        Dest_Addr_Countries.append("-")

    for ipindex in ipv4_df.index:
        while RIPE_HostAS_df.iat[ripeindex, 0] >= ipv4_df.at[ipindex, "ip_from"] and RIPE_HostAS_df.iat[ripeindex, 0] <= ipv4_df.at[ipindex, "ip_to"]:
            Dest_Addr_Countries.append(ipv4_df.at[ipindex, "country_code"])
            ripeindex = ripeindex + 1
            if ripeindex >= len(RIPE_HostAS_df[0]):
                break

        if ripeindex >= len(RIPE_HostAS_df[0]):
            break
            
    RIPE_HostAS_df["Country"] = Dest_Addr_Countries
    RIPE_HostAS_df = RIPE_HostAS_df.loc[RIPE_HostAS_df['Country'].isin(EU_list)]    
    
    
    #Add entries to complete dataframe
    frames = [Complete_RIPE_Entries_df, RIPE_HostAS_df]
    Complete_RIPE_Entries_df = pd.concat(frames)
    
    dur         = round(time.time() - start,2)
    print("Added " + str(len(RIPE_HostAS_df[0])) + " entries and " + str(len(unique_ASNs)) + " ASNs in " + str(dur) + " seconds")
    print()
    
    break

#Remove duplicates
Complete_ASN_Set = set(Complete_ASN_List)


D:/FoDa Data/ping-2022-03-01T0000.bz2
Lines added: 17099133Loading took: 531.3 seconds
Added 286373 entries and 113 ASNs in 584.04 seconds



In [79]:
Complete_ASN_Set_df = pd.DataFrame(Complete_ASN_Set)

Complete_ASN_Set_df.to_pickle("D:/FoDa Data/ASN_00.pkl")

Complete_RIPE_Entries_df.to_pickle("D:/FoDa Data/RIPE_00.pkl")

In [75]:
print(len(Complete_ASN_Set))
print(len(Complete_RIPE_Entries_df[0]))

113
237089


In [83]:
import ipaddress

int(ipaddress.IPv4Address('192.168.82.0'))

test_list = []

value = 10

for i in range(value):
    test = [i]
    test[0] = i 
    test.append(i) 
    test.append(i)
    test_list.extend(test)

print(test_list)

test_set = set(test_list)

print(test_set)
print()

test_total_df = pd.DataFrame({0:[],
                              "b":[]})
for i in range(value):
    test = {0: [i, i],
            "b": [(i*10),(i*10)]}
    test_df = pd.DataFrame(test)
    frames = [test_total_df, test_df]
    test_total_df = pd.concat(frames)

test_total_df

RIPE_Filenames = pd.date_range('2022-03-01', periods=6, freq='60min').strftime('D:/FoDa Data/ping-%Y-%m-%dT%H%M.bz2').tolist()
RIPE_Filenames

[0, 0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5, 5, 6, 6, 6, 7, 7, 7, 8, 8, 8, 9, 9, 9]
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}



['D:/FoDa Data/ping-2022-03-01T0000.bz2',
 'D:/FoDa Data/ping-2022-03-01T0100.bz2',
 'D:/FoDa Data/ping-2022-03-01T0200.bz2',
 'D:/FoDa Data/ping-2022-03-01T0300.bz2',
 'D:/FoDa Data/ping-2022-03-01T0400.bz2',
 'D:/FoDa Data/ping-2022-03-01T0500.bz2']

Description of results

## 2.4 Optimal server locations (Question E)
Since we are only allowed to place four servers, determine the best four datacenters based on the total
latency for all countries. Report your findings and your procedure to obtain them. Also include the
average latency for each country.


In [ ]:
#Code...

0Description of results

# Conclusions

... 
add code if needed

In [40]:
HostProbes = []

for ProbeASN in HostProbeASNs:
    
    index = 0
    for ASN in P_df["ASN"]:
        if ASN == ProbeASN:
            HostProbes.append(P_df["prb_id"][index])
            break
        index = index + 1
        
print(len(HostProbes))

194


In [41]:
decomFilename = 'C:/Users/Kooltje/Downloads/FoDa Data/ping-2022-03-01T2300'
decomFile     = open(decomFilename, 'rt')   


HostIPs = []
index = 0

for line in decomFile:
    jsonline = json.loads(line)
    
    
    if jsonline["prb_id"] in HostProbes:
        try:
            #Check for duplicates
            if jsonline["dst_addr"] not in HostIPs:
                #Check if IP is of type 4
                if jsonline["af"] == 4:
                    HostIPs.append(jsonline["dst_addr"])
        except KeyError as err:
            pass
    
    #Read only first 1m lines
    index = index + 1
    if index > 1000000:
        break
                              
print("Amount of IPs in the RIPE data connected to an AS of type Hosting: " + str(len(HostIPs)))

decomFile.close()

Amount of IPs in the RIPE data connected to an AS of type Hosting: 1047


In [46]:
#Converting the xxx.xxx.xxx.xxx format of the host IPs to integer
#Needed for when comparing to the IPv4 dataset

HostIPs_Integer = []


for IPString in HostIPs:
    IP_Splitstring = IPString.split(".") 
 
    HostIPs_Integer.append(int(IP_Splitstring[0]) * 16581375 + int(IP_Splitstring[1]) * 65025 + int(IP_Splitstring[2]) * 255 + int(IP_Splitstring[3]))  

print(len(HostIPs_Integer))



1047


In [20]:
#Upper part should be removed because run in part 1
import pickle

with open('data/AS_dataset.pkl', 'rb') as file:
    AS_df = pickle.load(file)
    
with open('data/probe_dataset.pkl', 'rb') as file:    
    P_df = pickle.load(file)
    
decomFilename = 'C:/Users/Kooltje/Downloads/FoDa Data/ping-2022-03-01T2300'
decomFile     = open(decomFilename, 'rt')   


RIPEProbes = []
index = 0

#Create list of all probes that are in the RIPE dataset
for line in decomFile:
    jsonline = json.loads(line)
    
    if jsonline["prb_id"] not in RIPEProbes:
        RIPEProbes.append(jsonline["prb_id"])
                          
    #Read only first 1m lines
    index = index + 1
    if index > 1000000:
        break
        
                  
print("Probes in first 1m lines of RIPE Dataset: " +str(len(RIPEProbes)))            

decomFile.close()


FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/Kooltje/Downloads/FoDa Data/ping-2022-03-01T2300'

In [38]:
index = 0
ProbeASNs = []

#Create list of all probes in both RIPE and probe datasets
#Saves only the ASNs as these are used later
#Probe IDs no longer used after this point
for probe in P_df["prb_id"]:
    if probe in RIPEProbes:
        ProbeASNs.append(P_df["ASN"][index])
        
    index = index + 1
    
print("Probes in both RIPE and probe dataset: " + str(len(ProbeASNs)))    

Probes in both RIPE and probe dataset: 7221


In [47]:
#List of country codes that are an EU member

EU_Countries = ["AT",
    "BE",
    "HR",
    "CY",
    "CZ",
    "DK",
    "EE",
    "FI",
    "FR",
    "GR",
    "DE",
    "HU",
    "IE",
    "IT",
    "LV",
    "LT",
    "LU",
    "MT",
    "NL",
    "PL",
    "PT",
    "RO",
    "SK",
    "SI",
    "ES",
    "SE"]

In [39]:
HostProbeASNs = []

index = 0

for ASN in AS_df["ASN"]:
    
    if ASN in ProbeASNs:
        if AS_df["type"][index] == "hosting":
            HostProbeASNs.append(ASN)            
    index = index + 1    
    
print("Amount of probes with an ASN with type hosting: " + str(len(HostProbeASNs))) 

HostProbeASNs.sort()


Amount of probes with an ASN with type hosting: 194


In [48]:
#Compare IPv4 with HostIPs_Integer

ipv4_df = pandas.read_csv("data/IP2LOCATION-LITE-DB1.CSV")
ipv4_df.rename(columns = {'0':'ip_from', '16777215':'ip_to',
                              '-':'country_code','-.1':'country_name'}, inplace = True)

ipv4_df.head()

HostIPs_EU = []
index = 0
ipv4index = 0

#Sorting the IP list so we can check from low to high IPs
HostIPs_Integer.sort

for IP_to in ipv4_df["ip_to"]:
    
    while HostIPs_Integer[index] < IP_to:
        if ipv4_df["country_code"][ipv4index] in EU_Countries:
            HostIPs_EU.append(HostIPs_Integer[index])
            print(HostIPs_Integer[index])
            print(ipv4_df["country_code"][ipv4index])
        index = index + 1
        if index >= len(HostIPs_Integer):
            break;
    ipv4index = ipv4index + 1  
    if index >= len(HostIPs_Integer):
        break;
                  

print(len(HostIPs_EU))


88072507
HU
1


In [ ]:
#create list of unique probe ID's in RIPE dataset
unique_prbID = []
for i in tuple_list:
    if i[1] not in unique_prbID:
        unique_prbID.append(i[1])
        
print("Unique probe IDs in RIPE dataset: " + str(len(unique_prbID.l)))

In [ ]:
index = 0
ProbeASNs = []

#Create list of all probes in both RIPE and probe datasets
#Saves only the ASNs as these are used later
#Probe IDs no longer used after this point
for probe in P_df["prb_id"]:
    if probe in RIPEProbes:
        ProbeASNs.append(P_df["ASN"][index])
        
    index = index + 1
    
print("Probes in both RIPE and probe dataset: " + str(len(ProbeASNs)))    

In [ ]:
HostProbes = []

index = 0

print(len(HostProbeASNs))

for ASN in P_df["ASN"]:
    if ASN in HostProbeASNs:
        if P_df["prb_id"][index] not in HostProbes:
            HostProbes.append(P_df["prb_id"][index])
    index = index + 1

print(len(HostProbes))    

In [ ]:
0#Random stuff I didn't want to throw away yet
#0Code for finding all host probes from EU in the dataset of one hour
import time
import bz2
import os
import sys
import json

# open decompressed file
decomFilename = 'C:/Users/Kooltje/Downloads/FoDa Data/ping-2022-03-01T2300'
decomFile     = open(decomFilename, 'rt') 

#read first line and print
#firstLine = decomFile.readline();
#print(firstLine)

#the line appears to be json-formatted: pretty print json
#firstLineJson = json.loads(firstLine)

#read all lines of first file
count = 0
st    = time.time()
for line in decomFile:
    jsonline = json.loads(line)
    #print(json.dumps(jsonline, sort_keys=True, indent=4))
    count = count + 1
    if count > 10000: 
          break

#print the last line
print(json.dumps(jsonline, sort_keys=True, indent=4))

#print the read duration
dur         = round(time.time() - st,2)
print("Loading took: " + str(dur) + " seconds")
print("The file had " + str(count) + "lines")

#finally close decomFile
decomFile.close()